In [2]:
# ANÁLISIS COMPLETO - LIMPIEZA Y VALIDACIÓN DE DATOS ESTUDIANTILES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("=" * 70)
print("ANÁLISIS COMPLETO - VALIDACIÓN Y LIMPIEZA DE DATOS")
print("=" * 70)

# 1. CARGA Y DIAGNÓSTICO INICIAL
print("\n1. DIAGNÓSTICO INICIAL DEL DATASET")
print("-" * 40)

# Simulamos un dataset con múltiples problemas
np.random.seed(42)
n = 400

data_problematico = {
    # Combina nombre, curso y edad en una sola columna separados por comas - viola principio una variable por columna
    'student_info': [f"Estudiante_{i},Curso_{np.random.choice(['A','B','C'])},Edad_{np.random.randint(15,19)}" for i in range(n)],
    
    # Combina horas de estudio y ausencias en texto separado por | - mezcla variables numéricas en string
    'studytime_absences': [f"{np.random.choice([1,2,3,4])}|{np.random.poisson(4)}" for _ in range(n)],
    
    # Primera evaluación con 10% de valores como 'N/A' y formato decimal - missing values incorrectos
    'G1': [f"{np.clip(np.random.normal(12,3), 0, 20):.1f}" if np.random.random() > 0.1 else 'N/A' for _ in range(n)],
    
    # Segunda evaluación con 15% de valores vacíos '' - inconsistencia en representación de missing values
    'G2': [f"{np.clip(np.random.normal(12,3), 0, 20):.1f}" if np.random.random() > 0.15 else '' for _ in range(n)],
    
    # Tercera evaluación como float sin formatear - inconsistencia de tipos con G1 y G2
    'G3': [np.clip(np.random.normal(12,3), 0, 20) for _ in range(n)],
    
    # Fechas como string en formato día/mes/año - dificulta ordenamiento y análisis temporal
    'fecha_evaluacion': [(datetime(2023,1,1) + timedelta(days=np.random.randint(0,365))).strftime('%d/%m/%Y') for _ in range(n)],
    
    # Categorización manual de notas - pérdida de información y límites arbitrarios
    'categoria_nota': ['Excelente' if x > 16 else 'Bueno' if x > 12 else 'Regular' if x > 8 else 'Insuficiente' for x in np.clip(np.random.normal(12,3,n), 0, 20)]
}


# Duplicamos algunas filas intencionalmente
filas_duplicadas = [data_problematico[col][:50] for col in data_problematico.keys()]
for i, col in enumerate(data_problematico.keys()):
    data_problematico[col].extend(filas_duplicadas[i])

df = pd.DataFrame(data_problematico)
print(f"Dataset original: {df.shape[0]} filas, {df.shape[1]} columnas")

# 2. ANÁLISIS DE ESTRUCTURA DE COLUMNAS
print("\n2. ANÁLISIS DE ESTRUCTURA DE DATOS")
print("-" * 40)

# 2.1 Verificar si los nombres de columnas son valores en lugar de variables
print(" Verificando estructura de columnas...")
columnas_sospechosas = []
for col in df.columns:
   
    if df[col].nunique() > len(df) * 0.8:
        columnas_sospechosas.append(col)
        print(f" Columna '{col}' podría contener datos en lugar de ser variable")
# 2.2 Detectar columnas con múltiples variables
print("\n Buscando columnas con múltiples variables...")
for col in df.columns:
    
    ejemplos = df[col].dropna().head(3).tolist()
    separadores = [',', '|', ';', '-', '/']
    
    for sep in separadores:
        if any(sep in str(ejemplo) for ejemplo in ejemplos if pd.notna(ejemplo)):
            print(f"    Columna '{col}' contiene múltiples variables separadas por '{sep}'")
            break  # Rompe el loop interno cuando encuentra el primer separador

# 3. LIMPIEZA DE DATOS COMPLETA
print("\n3. LIMPIEZA Y TRANSFORMACIÓN DE DATOS")
print("-" * 40)

# 3.1 Separar columnas con múltiples variables
print("Separando variables combinadas...")

df[['nombre_estudiante', 'curso', 'edad']] = df['student_info'].str.split(',', expand=True)
df['edad'] = df['edad'].str.replace('Edad_', '').astype(float)
df[['studytime', 'absences']] = df['studytime_absences'].str.split('|', expand=True)
df['studytime'] = pd.to_numeric(df['studytime'], errors='coerce')
df['absences'] = pd.to_numeric(df['absences'], errors='coerce')

# 3.2 Limpiar y convertir tipos de datos
print(" Corrigiendo tipos de datos...")

df['G1'] = df['G1'].replace(['N/A', '', 'NULL', 'null'], np.nan)  # Unifica missing values en G1
df['G2'] = df['G2'].replace(['', 'NULL', 'null'], np.nan)         # Estandariza valores nulos en G2

df['G1'] = pd.to_numeric(df['G1'], errors='coerce')  # Convierte texto en G1 a float/int, errores -> NaN
df['G2'] = pd.to_numeric(df['G2'], errors='coerce')  # Hace lo mismo en G2, garantizando uniformidad
df['G3'] = pd.to_numeric(df['G3'], errors='coerce')  # También en G3, para asegurar tipos de datos consistentes

# Convertir fecha
df['fecha_evaluacion'] = pd.to_datetime(df['fecha_evaluacion'], format='%d/%m/%Y', errors='coerce')

# 3.3 Eliminar columnas originales ya procesadas
columnas_eliminar = ['student_info', 'studytime_absences']
df = df.drop(columns=columnas_eliminar)

print(f"Nuevo dataset: {df.shape[0]} filas, {df.shape[1]} columnas")

# 4. DETECCIÓN Y TRATAMIENTO DE DATOS FALTANTES
print("\n4. ANÁLISIS DE DATOS FALTANTES")
print("-" * 40)

# 4.1 Análisis completo de valores faltantes
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (valores_faltantes / len(df)) * 100

print("Valores faltantes por columna:")
for col in df.columns:
    if valores_faltantes[col] > 0:
        print(f"   {col}: {valores_faltantes[col]} ({porcentaje_faltantes[col]:.1f}%)")

# 4.2 Estrategias de imputación según tipo de variable
print("\nAplicando estrategias de imputación...")

# Recorremos todas las columnas del DataFrame
for col in df.columns:
    # Verificamos si la columna tiene valores nulos
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['int64', 'float64']:
    
            if abs(df[col].skew()) > 2:  # skew mide la asimetría de la distribución
                impute_value = df[col].median()  # valor central menos afectado por outliers
                metodo = "mediana"
            else:
                impute_value = df[col].mean()    # promedio aritmético
                metodo = "media"

            # Rellenamos los nulos con el valor calculado
            df[col].fillna(impute_value, inplace=True)
            print(f"{col}: imputado con {metodo} ({impute_value:.2f})")

        #  2. Columnas Categóricas (tipo object)
         
        elif df[col].dtype == 'object':
            impute_value = df[col].mode()[0] if not df[col].mode().empty else 'Desconocido'
            df[col].fillna(impute_value, inplace=True)
            print(f"    {col}: imputado con moda ('{impute_value}')")

         
        # 3. Columnas de Fechas (datetime)
         
        elif 'datetime' in str(df[col].dtype):
            impute_value = df[col].mode()[0] if not df[col].mode().empty else df['fecha_evaluacion'].min()
            df[col].fillna(impute_value, inplace=True)
            print(f"    {col}: imputado con fecha más común")


print(f"Valores faltantes después de limpieza: {df.isnull().sum().sum()}")

# 5. DETECCIÓN Y ELIMINACIÓN DE DUPLICADOS
print("\n5. GESTIÓN DE DATOS DUPLICADO")
print("-" * 40)

# 5.1 Buscar duplicados exactos
duplicados_exactos = df.duplicated().sum()
print(f"Duplicados exactos encontrados: {duplicados_exactos}")

# 5.2 Buscar duplicados basados en clave natural (nombre + curso)
if 'nombre_estudiante' in df.columns and 'curso' in df.columns:
    duplicados_naturales = df.duplicated(subset=['nombre_estudiante', 'curso']).sum()
    print(f"Duplicados por estudiante-curso: {duplicados_naturales}")

# 5.3 Eliminar duplicados manteniendo el primer registro
filas_antes = len(df)
df = df.drop_duplicates()
if 'nombre_estudiante' in df.columns and 'curso' in df.columns:
    df = df.drop_duplicates(subset=['nombre_estudiante', 'curso'])
filas_despues = len(df)

print(f"Filas eliminadas por duplicación: {filas_antes - filas_despues}")
print(f"Dataset final: {filas_despues} filas")


ANÁLISIS COMPLETO - VALIDACIÓN Y LIMPIEZA DE DATOS

1. DIAGNÓSTICO INICIAL DEL DATASET
----------------------------------------
Dataset original: 450 filas, 7 columnas

2. ANÁLISIS DE ESTRUCTURA DE DATOS
----------------------------------------
 Verificando estructura de columnas...
 Columna 'student_info' podría contener datos en lugar de ser variable
 Columna 'G3' podría contener datos en lugar de ser variable

 Buscando columnas con múltiples variables...
    Columna 'student_info' contiene múltiples variables separadas por ','
    Columna 'studytime_absences' contiene múltiples variables separadas por '|'
    Columna 'fecha_evaluacion' contiene múltiples variables separadas por '/'

3. LIMPIEZA Y TRANSFORMACIÓN DE DATOS
----------------------------------------
Separando variables combinadas...
 Corrigiendo tipos de datos...
Nuevo dataset: 450 filas, 10 columnas

4. ANÁLISIS DE DATOS FALTANTES
----------------------------------------
Valores faltantes por columna:
   G1: 56 (12.4%)


C:\Users\Maeva\AppData\Local\Temp\ipykernel_17760\2962163341.py:138: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(impute_value, inplace=True)
